In [ ]:
import math
from sklearn.neighbors import KNeighborsClassifier
import keras
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KDTree
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
import cv2
import numpy as np
from matplotlib import pyplot as plt
import operator
from keras import *
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.layers import Activation, LeakyReLU
from tensorflow.keras.activations import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from keras.backend import abs, sum
from keras.preprocessing import image as kimage


In [ ]:
train_processing = keras.preprocessing.image.ImageDataGenerator(rotation_range=45, 
                                                                width_shift_range=0.1, 
                                                                height_shift_range=0.1, 
                                                                zoom_range=0.1, 
                                                                preprocessing_function=mobilenet_v2.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
train_generator= train_processing.flow_from_directory( 
    directory='mask_NEW_test_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 16,
    shuffle = True,
    seed = 1
)

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input)###resnet_v2 mobilenet_v2 vgg16
test_generator= test_processing.flow_from_directory(
    directory='mask_NEW_train_folders',
    target_size=(224,224), #scelta standard
    color_mode= 'rgb', #dobbiamo mettere stessi parametri della rete di partenza
    class_mode = 'categorical',
    batch_size = 16,
    shuffle = True,
    seed = 1
)

In [ ]:
base_net = mobilenet_v2.MobileNetV2(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net = resnet_v2.ResNet101V2(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net = vgg16.VGG16(input_shape=(224,224,3), weights ='imagenet', include_top=False, pooling='avg')

In [ ]:
base_net.summary()

In [ ]:
for layers in base_net.layers:
    layers.trainable = False

In [ ]:
x = base_net.output #base_net.get_layer('fc1').output

x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

pred = keras.layers.Dense(102, activation='softmax')(x) 


In [ ]:
x = base_net.output

x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

pred = keras.layers.Dense(102, activation='softmax')(x) 

In [ ]:
net = keras.Model(inputs=base_net.input, outputs=pred)

In [ ]:
net.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(), #keras.optimizers.RMSprop() #keras.optimizers.Adam() #sgd
            metrics=['accuracy'])

In [ ]:
history = net.fit(train_generator, batch_size=64, epochs=30, verbose=1, validation_data=test_generator) ####30

In [ ]:
net.save('mod3_100epochs_mobilenet.h5')

# PROVA PERFORMANCE

In [ ]:
net = None
net=keras.models.load_model('mod3_100epochs_mobilenet.h5')

In [ ]:
NAMES = [
    "pink primrose", "hard-leaved pocket orchid", "canterbury bells",
    "sweet pea", "english marigold", "tiger lily", "moon orchid",
    "bird of paradise", "monkshood", "globe thistle", "snapdragon",
    "colt's foot", "king protea", "spear thistle", "yellow iris",
    "globe-flower", "purple coneflower", "peruvian lily", "balloon flower",
    "giant white arum lily", "fire lily", "pincushion flower", "fritillary",
    "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers",
    "stemless gentian", "artichoke", "sweet william", "carnation",
    "garden phlox", "love in the mist", "mexican aster", "alpine sea holly",
    "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip",
    "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia",
    "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy",
    "common dandelion", "petunia", "wild pansy", "primula", "sunflower",
    "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia",
    "pink-yellow dahlia?", "cautleya spicata", "japanese anemone",
    "black-eyed susan", "silverbush", "californian poppy", "osteospermum",
    "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania",
    "azalea", "water lily", "rose", "thorn apple", "morning glory",
    "passion flower", "lotus", "toad lily", "anthurium", "frangipani",
    "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow",
    "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum",
    "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow",
    "mexican petunia", "bromelia", "blanket flower", "trumpet creeper",
    "blackberry lily"
]


In [ ]:
categories = {}
listimages = sorted(os.listdir('categories'))
for i in range(len(NAMES)):
    categories[NAMES[i]] = 'categories/' + listimages[i]

In [ ]:
def predict_name(image_path, classifier):
    img = kimage.load_img(image_path)
    x = kimage.img_to_array(img)
    x = mobilenet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis = 0)

    prediction = classifier.predict(x)

    return prediction[0], img

In [ ]:
def predict_name_photo (image_path, classifier):

    prediction, image = predict_name(image_path, classifier)
    prediction_name = NAMES[np.argmax(prediction)]

    path_pred_image = categories[prediction_name]
    image_pred = cv2.imread(path_pred_image)
    image_pred = cv2.cvtColor(image_pred, cv2.COLOR_BGR2RGB)
    
    NAMES.sort()

    plt.subplot(1,2,1),plt.imshow(image), plt.title('What is it?')
    plt.subplot(1,2,2),plt.imshow(image_pred), plt.title(prediction_name)

    #return NAMES[prediction-1]

In [ ]:
predict_name_photo('mask_NEW/image_07254.jpg', net)

In [ ]:
predict_name_photo('mask_NEW/image_06971.jpg', net)